In [1]:
import tobii_research as tr

def get_tracker():
  all_eyetrackers = tr.find_all_eyetrackers()

  for tracker in all_eyetrackers:
    print("Model: " + tracker.model)
    print("Serial number: " + tracker.serial_number) 
    print(f"Can stream eye images: {tr.CAPABILITY_HAS_EYE_IMAGES in tracker.device_capabilities}")
    print(f"Can stream gaze data: {tr.CAPABILITY_HAS_GAZE_DATA in tracker.device_capabilities}")
    return tracker

tracker = get_tracker()
tracker

Model: Tobii Pro Fusion
Serial number: TPFC6-010203104285
Can stream eye images: True
Can stream gaze data: True


In [9]:
import time


def gaze_data_callback(gaze_data):
  global global_gaze_data
  global_gaze_data = gaze_data
  
def gaze_data(eyetracker, wait_time=5):
  global global_gaze_data

  print("Getting data...")
  eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)

  time.sleep(wait_time)

  eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)

  return global_gaze_data
  
data = gaze_data(tracker)
data

Getting data...


{'device_time_stamp': 4487241857,
 'system_time_stamp': 223083126215,
 'left_gaze_point_on_display_area': (0.25351187586784363, 0.5755627751350403),
 'left_gaze_point_in_user_coordinate_system': (-46.50455856323242,
  55.14906692504883,
  9.43087387084961),
 'left_gaze_point_validity': 1,
 'left_pupil_diameter': 5.818115234375,
 'left_pupil_validity': 1,
 'left_gaze_origin_in_user_coordinate_system': (-55.121952056884766,
  12.225071907043457,
  539.22021484375),
 'left_gaze_origin_in_trackbox_coordinate_system': (0.6417078375816345,
  0.46810030937194824,
  0.21608063578605652),
 'left_gaze_origin_validity': 1,
 'right_gaze_point_on_display_area': (0.2533034384250641, 0.612749457359314),
 'right_gaze_point_in_user_coordinate_system': (-46.54388427734375,
  51.02855682373047,
  7.9310760498046875),
 'right_gaze_point_validity': 1,
 'right_pupil_diameter': 5.4077301025390625,
 'right_pupil_validity': 1,
 'right_gaze_origin_in_user_coordinate_system': (5.596215724945068,
  14.52007484436